## Анализ базы данных стартапа сервиса чтения книг по подписке

Задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


## Содержание проекта:
* 1 [Подключение к базе данных](#m1)
* 2 [Чтение таблиц из базы данных](#m2)
* 3 [Анализ данных](#m3)
    * 3.1 [Количество книг, выпущенных после 1 января 2000 года](#m3.1)
    * 3.2 [Количетво обзоров и средняя оценка книги](#m3.2)
    * 3.3 [Издатели с книгами 50+ страниц](#m3.3)
    * 3.4 [Автор с самой высокой оценкой (оценок 50+)](#m3.4)
    * 3.5 [Среднее количетсво обзоров от пользователей, которые поставили 50+ оценок](#m3.5)
* 4 [Общий вывод](#m4)

## Подключение к базе данных <a name="m1"></a>

In [1]:
# импортируем библиотеки
#!/usr/bin/python3
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Чтение таблиц из базы данных <a name="m2"></a>

In [2]:
books = ''' SELECT *
            FROM books
        '''
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
authors = ''' SELECT *
            FROM authors
        '''
pd.io.sql.read_sql(authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [4]:
publishers = ''' SELECT *
            FROM publishers
        '''
pd.io.sql.read_sql(publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [5]:
ratings = ''' SELECT *
            FROM ratings
        '''
pd.io.sql.read_sql(ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
reviews = ''' SELECT *
            FROM reviews
        '''
pd.io.sql.read_sql(reviews, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Анализ данных <a name="m2"></a>

### Количество книг, выпущенных после 1 января 2020 года <a name="m3.1"></a>

Работа с таблицей books

In [7]:
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [8]:
books_after_2000 = ''' SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(books_after_2000, con = engine) 

,count
0,819


819 книг в нашей таблице с данными выпущены после 1 января 2000 года.

### Количетво обзоров и средняя оценка книги <a name="m3.2"></a>

Для этой задачи используем таблицу ratings и reviews

pd.io.sql.read_sql(ratings, con = engine) 

In [12]:
mean_rating = '''SELECT
        ratings.book_id AS book_id,
        COUNT(reviews.text) AS total_reviews,
        AVG(ratings.rating) AS mean_rating
        FROM
            ratings
        LEFT JOIN reviews ON 
            (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
        GROUP BY 
            ratings.book_id
        ORDER BY 
            ratings.book_id
'''
pd.io.sql.read_sql(mean_rating, con = engine)

,book_id,total_reviews,mean_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


Ни одна книга не осталась без обзоров. Средний рейтинг книг в наших данных начинается с 1.5 и заканчивается 5.0. 

### Издатели с книгами 50+ страниц <a name="m3.3"></a>

Для этой задачи будем использовать две таблицы: publisher и books

In [11]:
page50 = ''' SELECT publisher, count(book_id) AS books_count
             FROM (SELECT books.*, publishers.publisher
             FROM books, publishers
             WHERE books.publisher_id = publishers.publisher_id) AS tbl
             WHERE num_pages > 50
             GROUP BY publisher
             ORDER BY books_count DESC 
             LIMIT 1
'''

pd.io.sql.read_sql(page50, con = engine) 

,publisher,books_count
0,Penguin Books,42


Издательство penguin books выпустило больше всего книг с 50+ страницами

### Автор с самой высокой оценкой (оценок 50+) <a name="m3.4"></a>

In [14]:
popular_author = '''SELECT authors.author, avg(book_avg_ratings.book_rating_avg) as author_rating_avg
                    FROM books
                    INNER JOIN (SELECT book_id, avg(rating) as book_rating_avg
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(*) >= 50) AS book_avg_ratings
                    ON book_avg_ratings.book_id = books.book_id
                    INNER JOIN authors ON authors.author_id = books.author_id
                    GROUP BY authors.author
                    ORDER BY author_rating_avg desc
                    
'''

pd.io.sql.read_sql(popular_author, con = engine)

,author,author_rating_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самый высокооцененный автор с 50+ оценками - это Роулинг, автор книг по Гарри Поттеру.

### Среднее количетсво обзоров от пользователей, которые поставили 50+ оценок <a name="m3.4"></a>

In [13]:
users = '''SELECT AVG(tbl.count)
           FROM (SELECT total.username, COUNT(total.username)
                 FROM (SELECT username, COUNT(rating_id) AS count
                       FROM ratings
                       GROUP BY username) as total
                 JOIN reviews ON reviews.username = total.username
                 WHERE total.count > 50
                 GROUP BY total.username) as tbl
            
'''

pd.io.sql.read_sql(users, con = engine)

,avg
0,24.333333


Активные на оценки пользователи в среднем пишут 24 оценки. То есть примерно каждая вторая книга обозревается активным пользователем.

## Общий вывод <a name="m4"></a>

На основы исследования можно сделать следующие выводы:
* 819 книг выпущено после 1 января 2000 года
* Ни одна книга не осталась без обзора, средняя оценки книг начинается с 1.5 и заканчивается 5.0
* Penguin Books - издательство, которое выпустило больше всего книг с 50+ страницами
* Роулинг имеет самую высокую среднюю оценку как автор: 4.3
* Активные пользователи, которые поставили 50+ оценок, в среднем пишут 24 обзора, то есть обозревают почти каждую вторую книгу